### CODE

In [3]:
from z3 import *
import random

def calculate_high_consumption_goods_tax(item_type, base_amount):
    """
    計算高額消費貨物的應納稅額。
    
    參數：
      - item_type: 貨物種類（例如 "小客車"、"遊艇"、"飛機/直昇機/超輕型載具"、"龜殼等"、"家具"）。
                   註：這些項目皆依【特種貨物及勞務稅條例第 7 條】，以銷售價格或完稅價格乘以10%計算應納稅額。
      - base_amount: 貨物的銷售價格或完稅價格（數值）。
    
    回傳：
      應納稅額的整數部分（例如 300000 表示 300,000 元）。
    """
    s = Solver()
    # 將輸入的基礎金額建立為 Z3 常數
    A = RealVal(base_amount)
    s.add(A == base_amount)
    
    # 應納稅額 = 基礎金額 × 10%
    tax_expr = A * 0.10
    # 取整數部分
    tax_int_expr = ToInt(tax_expr)
    
    if s.check() == sat:
        m = s.model()
        tax_int = m.evaluate(tax_int_expr, model_completion=True).as_long()
        return tax_int
    else:
        return None

def calculate_high_consumption_services_tax(sales_price):
    """
    計算高額消費勞務（入會權利）的應納稅額。
    
    參數：
      - sales_price: 該次銷售（或完稅）價格（數值）。
                   註：根據【特種貨物及勞務稅條例第 7 條】，稅率為10%，
                   此處假設銷售價格已符合課稅範圍（不包括可退還性保證金）。
    
    回傳：
      應納稅額的整數部分（例如 50000 表示 50,000 元）。
    """
    s = Solver()
    A = RealVal(sales_price)
    s.add(A == sales_price)
    
    tax_expr = A * 0.10
    tax_int_expr = ToInt(tax_expr)
    
    if s.check() == sat:
        m = s.model()
        tax_int = m.evaluate(tax_int_expr, model_completion=True).as_long()
        return tax_int
    else:
        return None

# 示範如何呼叫
if __name__ == '__main__':
    # 例如：計算「小客車」的高額消費貨物稅，
    # 若其銷售價格或完稅價格為 3,000,000 元，則應納稅額 = 3,000,000 × 10% = 300,000 元
    goods_tax = calculate_high_consumption_goods_tax("小客車", 3000000)
    print("高額消費貨物稅（小客車）：", goods_tax)
    
    # 例如：計算高額消費勞務（入會權利）的稅額，
    # 若銷售價格為 500,000 元，則應納稅額 = 500,000 × 10% = 50,000 元
    services_tax = calculate_high_consumption_services_tax(500000)
    print("高額消費勞務稅（入會權利）：", services_tax)


高額消費貨物稅（小客車）： 300000
高額消費勞務稅（入會權利）： 50000


In [5]:
def generate_random_test_parameters():
    # 定義各貨物項目的隨機產生範圍
    goods_item_types = {
        "小客車": (3000000, 10000000),   # 小客車：最低 3,000,000
        "遊艇": (500000, 2000000),         # 遊艇：最低 500,000
        "飛機/直昇機/超輕型載具": (3000000, 10000000),
        "龜殼等": (500000, 2000000),        # 龜殼、玳瑁等：最低 500,000
        "家具": (500000, 2000000)           # 家具：最低 500,000
    }
    # 高額消費勞務 (入會權利) 的銷售價格範圍
    services_range = (500000, 2000000)
    
    print("===== 高額消費貨物測試參數 =====")
    for i in range(5):
        # 隨機選擇一個貨物項目
        item_type = random.choice(list(goods_item_types.keys()))
        rmin, rmax = goods_item_types[item_type]
        base_amount = random.randint(rmin, rmax)
        expected_tax = calculate_high_consumption_goods_tax(item_type, base_amount)
        print(f"Test {i+1:03d}: {item_type}, base_amount = {base_amount}, expected_tax = {expected_tax}")
    
    print("\n===== 高額消費勞務測試參數 =====")
    for i in range(5):
        sales_price = random.randint(services_range[0], services_range[1])
        expected_tax = calculate_high_consumption_services_tax(sales_price)
        print(f"Test {i+1:03d}: sales_price = {sales_price}, expected_tax = {expected_tax}")

if __name__ == '__main__':
    generate_random_test_parameters()

===== 高額消費貨物測試參數 =====
Test 001: 小客車, base_amount = 3742004, expected_tax = 374200
Test 002: 飛機/直昇機/超輕型載具, base_amount = 6959364, expected_tax = 695936
Test 003: 飛機/直昇機/超輕型載具, base_amount = 3858694, expected_tax = 385869
Test 004: 遊艇, base_amount = 501086, expected_tax = 50108
Test 005: 龜殼等, base_amount = 1897962, expected_tax = 189796

===== 高額消費勞務測試參數 =====
Test 001: sales_price = 1443413, expected_tax = 144341
Test 002: sales_price = 1210250, expected_tax = 121025
Test 003: sales_price = 1014878, expected_tax = 101487
Test 004: sales_price = 1119318, expected_tax = 111931
Test 005: sales_price = 1214199, expected_tax = 121419
